In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/icfes-colombia-20182021/icfes_data.csv', sep='\t', engine='python')

In [ ]:
df.sample(10)

# EDA

In [ ]:
df['PUNT_GLOBAL'].hist(); plt.title('Global Score Histogram'); plt.show()
print('% of women: ', 100*(df['ESTU_GENERO']=='F').sum() / df.shape[0], '%')

## Student

In [ ]:
subject_cols = [i for i in df.columns if i.startswith('PUNT_')]
subject_cols.remove('PUNT_GLOBAL')
df.groupby('ESTU_GENERO').mean()[subject_cols].transpose().plot.bar()
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.title('Subject Score by Gender'); plt.show()

In [ ]:
df['ESTU_FECHANACIMIENTO'] = pd.to_datetime(df['ESTU_FECHANACIMIENTO'], dayfirst=True, errors='coerce')
today = pd.to_datetime('today')
df['STUDENT_AGE'] = today.year - df['ESTU_FECHANACIMIENTO'].dt.year

df.groupby(pd.cut(df['STUDENT_AGE'], bins=[0,15,18,30,50,200])).mean()[subject_cols].transpose().plot.bar()
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.title('Subject Scores by Age'); plt.show()

## Scores 

In [ ]:
subject_df = df[subject_cols]
subject_z_score = (subject_df - subject_df.mean()) / subject_df.std().std()
subject_z_score.corr()

## Family

In [ ]:
food_cols = ['FAMI_COMELECHEDERIVADOS','FAMI_COMECARNEPESCADOHUEVO','FAMI_COMECEREALFRUTOSLEGUMBRE']
food_dict = {
 'Nunca o rara vez comemos eso': 0,
 '1 o 2 veces por semana': 1,
 '3 a 5 veces por semana': 2,
 'Todos o casi todos los días': 3
}
df['_food_score'] = df[food_cols].replace(food_dict).mean(axis=1)

In [ ]:
df.groupby(pd.cut(df['_food_score'], bins=5))[subject_cols].mean().transpose().plot.bar()
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.title('Subject Score by Food Score'); plt.show()